In [21]:
from databricks import sql
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import date, datetime, timedelta
from win32com import client
import xlwings as xw
from selenium import webdriver
import time
import shutil

In [ ]:
with open('APIKEYS.txt', 'r') as f: 
    # Read the contents of the file into a list 
    lines = f.readlines() 
    # Create an empty dictionary 
    APIKEYS = {} 
    # Loop through the list of lines 
    for line in lines: 
        # Split the line into key-value pairs 
        key, value = line.strip().split(',') 
        # Store the key-value pairs in the dictionary 
        APIKEYS[key] = value 
# The dictionary 'data' now contains the contents of the text file 

In [22]:
with open(f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF.txt', 'r') as f: 
    # Read the contents of the file into a list 
    lines = f.readlines() 
    # Create an empty dictionary 
    LastUpdate = {} 
    # Loop through the list of lines 
    for line in lines: 
        # Split the line into key-value pairs 
        key, value = line.strip().split(',') 
        # Store the key-value pairs in the dictionary 
        LastUpdate[key] = value 
# The dictionary 'data' now contains the contents of the text file 


In [23]:
#get yesterdays date (for query)
yesterday = datetime.now() - timedelta(1)
yesterday_str = yesterday.strftime('%Y-%m-%d')


In [24]:


# Get Parcel data from DB and place in table called "Parcel_result"
with sql.connect(server_hostname = APIKEYS['server_hostname'],
                 http_path       = APIKEYS['http_path'],
                 access_token    = APIKEYS['access_token']) as connection:

  with connection.cursor() as cursor:
    cursor.execute(f"""select 

OP.*, 
Case 
when op.Customer_Service_Result like '%Not Despatched On Time%' or op.Customer_Service_Result like '%Not Despatched on Time%' then op.Customer_Service_Result
when op.Customer_service_result = 'Other - check logic' then 'Carrier Fail'
when lup.OTIF_Category is null then 'HELP' 
else lup.OTIF_Category end as OTIF_New_Result

from WhseFin_SharedArea.Site_OTIF_Parcels_New as OP
Left Join warehouse_reporting.OTIF_Lookup as Lup on OP.DeliveryMethod = Lup.MOD and OP.Customer_Service_Result = Lup.Customer_Service_Result

where  promise_date between '{yesterday_str}' and '{yesterday_str}'""")
    Parcel_result = cursor.fetchall()

#Get OTIF results from DB and place in table called "OTIF_result"
with sql.connect(server_hostname = APIKEYS['server_hostname'],
                 http_path       = APIKEYS['http_path'],
                 access_token    = APIKEYS['access_token']) as connection:

  with connection.cursor() as cursor:
    cursor.execute(f"""Select 

O.*,
Case 
when o.Customer_Service_Result like '%Not Despatched On Time%' or o.Customer_Service_Result like '%Not Despatched on Time%' then o.Customer_Service_Result
when o.Customer_service_result = 'Other - check logic' then 'Carrier Fail'
when lup.OTIF_Category is null then 'HELP' 
else lup.OTIF_Category end as OTIF_New_Result

from WhseFin_SharedArea.OTIF_Last_14 as O
Left Join warehouse_reporting.OTIF_Lookup as Lup on O.MOD = Lup.MOD and O.Customer_Service_Result = Lup.Customer_Service_Result

Where O.MOD <> 'Other'
and promise_date between '{LastUpdate['LastUpdatedDate']}' and '{yesterday_str}' 

order by promise_date desc""")
    OTIF_result = cursor.fetchall()



In [25]:

#convert tables to pandas, listing all columns
All_data = pd.DataFrame(Parcel_result, columns = ['ParcelBarcode',	'Parcel_barcode',	'Generation_date',	'account_no',	'order_date',	'Promise_date',	'whse',	'MOD',	'Batch',	'NoRecord',	'ItemCancelled',	'NoStocked',	'ItemDespatchedOnTime',	'ItemDespatchedLate',	'Total_Items',	'depot',	'packing_station',	'parcel_collate_status',	'grand_collation_flag',	'WarehouseCheck',	'Hub_Parcel',	'Latest_Status',	'Latest_status_date',	'Latest_Status_time',	'On_Way_To_Courier_Date',	'On_Way_To_Courier_Time',	'Last_Status_Promise_Date',	'Last_Status_Promise_Time',	'Promise_Date_description',	'Status_Code_Promise',	'Sub_Status_Promise',	'Classification',	'FileDateTime',	'FileDateYear',	'FileDateDay',	'FileDateMonth',	'DeliveryMethod',	'Warehouse_Despatch_Result',	'Customer_Service_Result',	'OTIF_New_Result'])
OTIF_result = pd.DataFrame(OTIF_result, columns = ['Whse',	'MOD',	'Promise_date',	'Week',	'Season',	'DOW',	'Warehouse_Despatch_Result',	'Customer_Service_Result',	'Parcels',	'No_record_of_despatch_items',	'No_Stocked_Items',	'Item_despatched_on_time',	'Item_Despatched_late',	'Total_Items',	'OTIF_New_Result'])

In [26]:
OTIF_result

,Whse,MOD,Promise_date,Week,Season,DOW,Warehouse_Despatch_Result,Customer_Service_Result,Parcels,No_record_of_despatch_items,No_Stocked_Items,Item_despatched_on_time,Item_Despatched_late,Total_Items,OTIF_New_Result
0,Belfast,NDTS,2023-04-23,13,SS2023,1,On Time in Full,Delivered on Time,12,0,0,28,0,28,Delivered On Time
1,Belfast,NDTS,2023-04-23,13,SS2023,1,On Time in Full,Delivered on Time,827,0,0,1548,0,1548,Delivered On Time
2,Belfast,NDTS,2023-04-23,13,SS2023,1,On Time in Full,Delivered on Time,8,0,0,25,0,25,Delivered On Time
3,Belfast,NDTS,2023-04-23,13,SS2023,1,On Time in Full,Delivered on Time,1502,0,0,2715,0,2715,Delivered On Time
4,Belfast,NDTS,2023-04-23,13,SS2023,1,Packed & Despatched Late,Not Despatched On Time,12,0,0,14,0,14,Not Despatched On Time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,VS - PSS,NDTS,2023-04-21,12,SS2023,6,On Time in Full,Delivered on Time,4,0,0,4,0,4,Delivered On Time
10862,VS - PSS,Hermes,2023-04-21,12,SS2023,6,On Time in Full,Delivered on Time,2,0,0,3,0,3,Delivered on Time
10863,VS - PSS,NDTS,2023-04-21,12,SS2023,6,On Time in Full,Delivered on Time,2,0,0,3,0,3,Delivered On Time
10864,VS - PSS,Hermes,2023-04-21,12,SS2023,6,On Time in Full,Hermes Fail - Clock Stop Event,1,0,0,1,0,1,Carrier Fail


In [27]:
#Get current week number
curr_week = OTIF_result.iloc[0][3].astype('str')
#Get current week number
curr_season_full = OTIF_result.iloc[0][4]

curr_season = curr_season_full[0:2] + curr_season_full[4:6]

curr_year_full = curr_season_full[2:6]

In [28]:

#any datetime columns need to be turned to date
All_data['FileDateTime'] = pd.to_datetime(All_data['FileDateTime']).dt.date

In [29]:

#Create the separate tables for the parcel data files
Elmsall = All_data[All_data['whse'] == 'Elmsall']
DVP_And_VNA = All_data[(All_data['whse'] == 'DVP' ) | (  All_data['whse'] == 'VNA')]
Stadium_Fails = All_data[(All_data['whse'] == 'Stadium' ) & (All_data['OTIF_New_Result'] != 'Delivered On Time') & (All_data['OTIF_New_Result'] != 'Delivered on Time')]
Reiss_Fails = All_data[(All_data['whse'] == 'Reiss' ) & (All_data['OTIF_New_Result'] != 'Delivered On Time') & (All_data['OTIF_New_Result'] != 'Delivered on Time')]
Doncaster_Fails = All_data[(All_data['whse'] == 'Doncaster' ) & (All_data['OTIF_New_Result'] != 'Delivered On Time') & (All_data['OTIF_New_Result'] != 'Delivered on Time')]
Redhouse = All_data[All_data['whse'] == 'Redhouse']
GAP = All_data[All_data['whse'] == 'GAP']
JoJo = All_data[All_data['whse'] == 'Jojo']

In [30]:
#Get current date to save files with a dated name
today = date.today()
date = today.strftime("%d.%m.%Y")
CurrentDateString = today.strftime("%Y-%m-%d")

In [31]:
#create and save the files
Elmsall.to_excel('{0} - Elmsall.xlsx'.format(date), sheet_name='Data',index = False)
DVP_And_VNA.to_excel('{0} - DVP_And_VNA.xlsx'.format(date), sheet_name='Data',index = False)
Stadium_Fails.to_excel('{0} - Stadium_Fails.xlsx'.format(date), sheet_name='Data',index = False)
Reiss_Fails.to_excel('{0} - Reiss_Fails.xlsx'.format(date), sheet_name='Data',index = False)
Doncaster_Fails.to_excel('{0} - Doncaster_Fails.xlsx'.format(date), sheet_name='Data',index = False)
Redhouse.to_excel('{0} - Redhouse.xlsx'.format(date), sheet_name='Data',index = False)
GAP.to_excel('{0} - GAP.xlsx'.format(date), sheet_name='Data',index = False)
JoJo.to_excel('{0} - JoJo.xlsx'.format(date), sheet_name='Data',index = False)

In [32]:

def Update():

    if LastUpdate['LastUpdatedSeason'] == curr_season:
        #Create app with visible = False to make below run in the background (saves memory and time)
        app = xw.App(visible=False)

        #Select File and sheet to update
        wb = app.books.open(f"C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {LastUpdate['LastFileWeek']}.xlsx")
        ws = wb.sheets["Data"]

        #find which row we need to place data
        last_row = ws.range(1,1).end('down').row
        Import_row = last_row + 1

        #Place the results Data
        ws.range("A{0}".format(Import_row)).options(index=False).value = OTIF_result

        #save the file
        wb.save(path=f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {curr_week}.xlsx')

        #stop the app
        app.kill()
    elif LastUpdate['LastUpdatedSeason'] != curr_season:
        #Create app with visible = False to make below run in the background (saves memory and time)
        app = xw.App(visible=False)

        #Select File and sheet to update
        wb = app.books.open(f"C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {LastUpdate['LastFileWeek']}.xlsx")
        ws = wb.sheets["Data"]

        #find which row we need to place data
        last_row = ws.range(1,1).end('down').row
        Import_row = last_row + 1

        #Clear Contents for new season
        ws.range(f"A2:O{last_row}").clear_contents()

        #Place the results Data
        ws.range("A{0}".format(Import_row)).options(index=False).value = OTIF_result

        #save the file
        wb.save(path=f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {curr_week}.xlsx')

        #stop the app
        app.kill()

Update()

In [33]:

#get graphs page pdf 
excel = client.Dispatch("Excel.Application")
  
# Read Excel File
sheets = excel.Workbooks.Open(f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {curr_week}.xlsx')
work_sheets_week = sheets.Worksheets['Graphs']
  
# Convert into PDF File
work_sheets_week.ExportAsFixedFormat(0, f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {curr_week}.pdf')


sheets.Close(SaveChanges=0)
excel.Quit()

In [34]:

#get summary page pdf

excel = client.Dispatch("Excel.Application")
  
# Read Excel File
sheets = excel.Workbooks.Open(f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {curr_week}.xlsx')
work_sheets_week = sheets.Worksheets['Email']
  
# Convert into PDF File
work_sheets_week.ExportAsFixedFormat(0, f'C:\\Users\\{os.getlogin()}\\Desktop\\OTIFS\\UK\\UK OTIF - Summary - Week {curr_week}.pdf',IgnorePrintAreas = False)


sheets.Close(SaveChanges=0)
excel.Quit()

In [35]:
#In or out of office 
def OfficeCheck():
    if os.path.exists('T:\\Common\\Management Information') == False:
        return 'Home'
    elif os.path.exists('T:\\Common\\Management Information') == True:
        return 'Office'


In [36]:
OfficeFile = 'T:\\Common\Management Information\\On Time in Full\\UK OTIF\\{season}\\Week {week}\\UK OTIF - Week {week}.xlsx'
DesktopLoc = 'C:\\Users\\{User}\\Desktop\\OTIFS\\UK\\UK OTIF - Week {week}.xlsx'
OfficeLoc =  'T:\\Common\Management Information\\On Time in Full\\UK OTIF\\{season}\\Week {week}'


In [37]:
LastUpdate

{'LastFileWeek': '12',
 'LastUpdatedDate': '2023-04-21',
 'LastUpdatedSeason': 'SS23',
 'LastUpdatedLoc': 'Home',
 'LastUpdatedOfficeDate': '2023-04-20',
 'LastUpdatedHomeDate': '2023-04-21',
 'LastUpdatedOfficeSeason': 'SS23',
 'LastUpdatedHomeSeason': 'SS23',
 'LastFileOfficeWeek': '12',
 'LastFileHomeWeek': '12'}

In [38]:
def Save(DesktopLoc,OfficeLoc,OfficeFile,LastUpdate):
    #If at home, update as normal
    if OfficeCheck() == 'Home':
        return

    #If in Office and the last update was also in the office, save the latest copy in MI
    elif OfficeCheck() == 'Office' and LastUpdate['LastUpdatedDate'] == LastUpdate['LastUpdatedOfficeDate']:

        #Create file path if not exists for recent update
        if not os.path.exists(OfficeLoc.format(season = curr_season,week = curr_week)):
            os.mkdir(OfficeLoc.format(season = curr_season,week = curr_week))
            
        #copy latest file to MI from desktop
        shutil.copyfile(DesktopLoc.format(week = curr_week,User = os.getlogin()),OfficeFile.format(season = curr_season, week = curr_week))

        #remove last weeks file if in new week
        if  LastUpdate['LastFileOfficeWeek'] != curr_week:
            os.remove(DesktopLoc.format(week = LastUpdate['LastFileOfficeWeek'],User = os.getlogin()))    
    
    #If in office and last update was at home
    elif OfficeCheck() == 'Office' and LastUpdate['LastUpdatedDate'] == LastUpdate['LastUpdatedHomeDate']:

        #Still the same week so just need to check file path exists and save file
        if LastUpdate['LastFileHomeWeek'] == LastUpdate['LastFileWeek']:
            if not os.path.exists(OfficeLoc.format(season = curr_season,week = curr_week)):
                os.mkdir(OfficeLoc.format(season = curr_season, week = curr_week))
            shutil.copyfile(DesktopLoc.format(week = curr_week,User = os.getlogin()),OfficeFile.format(season = curr_season, week = curr_week))

        #If we are not in the same week, we need to save last weeks file down and this weeks file into the correct paths
        elif LastUpdate['LastFileWeek'] != LastUpdate['LastFileHomeWeek']:

            #save last weeks file
            if not os.path.exists(OfficeLoc.format(season = LastUpdate['LastUpdatedHomeSeason'], week = LastUpdate['LastFileHomeWeek'])):
                os.mkdir(OfficeLoc.format(season = LastUpdate['LastUpdatedHomeSeason'], week = LastUpdate['LastFileHomeWeek']))
            shutil.copyfile(DesktopLoc.format(week = LastUpdate['LastFileHomeWeek'],User = os.getlogin()),OfficeFile.format(season = LastUpdate['LastUpdatedHomeSeason'], week = LastUpdate['LastFileHomeWeek']))

            #save this weeks file
            if not os.path.exists(OfficeLoc.format(season = curr_season,week = curr_week)):
                os.mkdir(OfficeLoc.format(season = curr_season, week = curr_week))
            shutil.copyfile(DesktopLoc.format(week = curr_week,User = os.getlogin()),OfficeFile.format(season = curr_season, week = curr_week))

            #remove last weeks file from desktop to save space
            os.remove(DesktopLoc.format(week = LastUpdate['LastFileHomeWeek'],User = os.getlogin()))

Save(DesktopLoc,OfficeLoc,OfficeFile,LastUpdate)

In [39]:
# Read the contents of the file into memory
with open('C:\\Users\\CFPRICE\\Desktop\\OTIFS\\UK\\UK OTIF.txt', 'r') as f:
    lines = f.readlines()

# Update the values in memory
for i, line in enumerate(lines):
    key, value = line.strip().split(',')
    if key == 'LastFileWeek':
        value = str(curr_week)
    if key == 'LastUpdatedDate':
        value = str(CurrentDateString)# update the values
    if key == 'LastUpdatedLoc':
        value = OfficeCheck()
    if key == 'LastUpdatedSeason':
        value = curr_season
    if key == f'LastUpdated{OfficeCheck()}Date':
        value = str(CurrentDateString)
    if key == f'LastFile{OfficeCheck()}Week':
        value = str(curr_week)
    if key == f'LastUpdated{OfficeCheck()}Season':
        value = curr_season
    lines[i] = key + ',' + value + '\n'
    

with open('C:\\Users\\CFPRICE\\Desktop\\OTIFS\\UK\\UK OTIF.txt', 'w') as f:
    f.writelines(lines)